#Importing Libraries

In [ ]:
import pandas as pd
import numpy as np
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("bhavikjikadara/fake-news-detection")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/fake-news-detection


In [ ]:
fake = pd.read_csv(path + "/fake.csv")
fake.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [ ]:
real = pd.read_csv(path + "/true.csv")
real.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [ ]:
fake.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23481 entries, 0 to 23480
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    23481 non-null  object
 1   text     23481 non-null  object
 2   subject  23481 non-null  object
 3   date     23481 non-null  object
dtypes: object(4)
memory usage: 733.9+ KB


In [ ]:
real.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21417 entries, 0 to 21416
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    21417 non-null  object
 1   text     21417 non-null  object
 2   subject  21417 non-null  object
 3   date     21417 non-null  object
dtypes: object(4)
memory usage: 669.4+ KB


In [ ]:
fake['label'] = 0
real['label'] = 1

In [ ]:
df = pd.concat([fake, real], axis = 0)
df

,title,text,subject,date,label
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0
...,...,...,...,...,...
21412,'Fully committed' NATO backs new U.S. approach...,BRUSSELS (Reuters) - NATO allies on Tuesday we...,worldnews,"August 22, 2017",1
21413,LexisNexis withdrew two products from Chinese ...,"LONDON (Reuters) - LexisNexis, a provider of l...",worldnews,"August 22, 2017",1
21414,Minsk cultural hub becomes haven from authorities,MINSK (Reuters) - In the shadow of disused Sov...,worldnews,"August 22, 2017",1
21415,Vatican upbeat on possibility of Pope Francis ...,MOSCOW (Reuters) - Vatican Secretary of State ...,worldnews,"August 22, 2017",1


In [ ]:
df = df.sample(frac = 1).reset_index(drop = 1)
df

,title,text,subject,date,label
0,WOW! NATIONAL GEOGRAPHIC Gets Hammered With EX...,"Because by 9 years old, pretty much every pers...",left-news,"Dec 19, 2016",0
1,"TRUMP ARRIVES IN MOBILE, ALABAMA TO SAY THANKS...",,politics,"Dec 17, 2016",0
2,Trump to drop climate change from environmenta...,(Reuters) - President Donald Trump is set to s...,politicsNews,"March 14, 2017",1
3,HOW MEMPHIS TEEN DEFIED OBAMA’S “Black Victim”...,"After 8 brutal years of race-baiting, endless ...",politics,"Sep 9, 2016",0
4,The New American Century: An Era of Fraud,Paul Craig RobertsIn the last years of the 20t...,US_News,"January 19, 2016",0
...,...,...,...,...,...
44893,Turkish army expands deployment in Syria's nor...,AMMAN (Reuters) - The Turkish army is expandin...,worldnews,"October 15, 2017",1
44894,Late UK PM Heath had questions to answer over ...,"SWINDON, England (Reuters) - Former British pr...",worldnews,"October 5, 2017",1
44895,Qatar enacts law to protect foreign domestic w...,DOHA (Reuters) - Qatar s ruler has introduced ...,worldnews,"August 24, 2017",1
44896,HYSTERICAL! TRUMP LIFE: “There ain’t a brother...,#SourcesHaveConfirmed that Trump will lock her...,politics,"Nov 5, 2016",0


In [ ]:
df['text'] = df.title + ' ' + df.text
df.drop(['title'], axis = 1, inplace = True)

In [ ]:
df.head()

,text,subject,date,label
0,WOW! NATIONAL GEOGRAPHIC Gets Hammered With EX...,left-news,"Dec 19, 2016",0
1,"TRUMP ARRIVES IN MOBILE, ALABAMA TO SAY THANKS...",politics,"Dec 17, 2016",0
2,Trump to drop climate change from environmenta...,politicsNews,"March 14, 2017",1
3,HOW MEMPHIS TEEN DEFIED OBAMA’S “Black Victim”...,politics,"Sep 9, 2016",0
4,The New American Century: An Era of Fraud Paul...,US_News,"January 19, 2016",0


In [ ]:
df.drop(['date', 'subject'], axis = 1, inplace = True)

In [ ]:
x = df.drop('label', axis = 1)
y = df['label']

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
print(stopwords.words('english'))

['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', "he'd", "he'll", 'her', 'here', 'hers', 'herself', "he's", 'him', 'himself', 'his', 'how', 'i', "i'd", 'if', "i'll", "i'm", 'in', 'into', 'is', 'isn', "isn't", 'it', "it'd", "it'll", "it's", 'its', 'itself', "i've", 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'shan', "shan't", 'she

In [ ]:
port_stem = PorterStemmer()

In [ ]:
def stemming(content):
  steammed_content = re.sub('[^a-zA-Z]',' ',content)
  steammed_content = steammed_content.lower()
  steammed_content = steammed_content.split()
  steammed_content = [port_stem.stem(word) for word in steammed_content if not word in stopwords.words('english')]
  steammed_content = ' '.join(steammed_content)
  return steammed_content

In [ ]:
df['text'] = df['text'].apply(stemming)

In [ ]:
 df.head()

,text,label
0,wow nation geograph get hammer explos reaction...,0
1,trump arriv mobil alabama say thank obama vaca...,0
2,trump drop climat chang environment review blo...,1
3,memphi teen defi obama black victim narr use p...,0
4,new american centuri era fraud paul craig robe...,0


In [ ]:
print(df['text'])

0        wow nation geograph get hammer explos reaction...
1        trump arriv mobil alabama say thank obama vaca...
2        trump drop climat chang environment review blo...
3        memphi teen defi obama black victim narr use p...
4        new american centuri era fraud paul craig robe...
                               ...                        
44893    turkish armi expand deploy syria northwest reb...
44894    late uk pm heath question answer child sex abu...
44895    qatar enact law protect foreign domest worker ...
44896    hyster trump life brother bigger stronger gang...
44897    hyster nanci pelosi fellow dem clueless mic wo...
Name: text, Length: 44898, dtype: object


In [ ]:
x = df['text'].values
y = df['label'].values

In [ ]:
vectorizer = TfidfVectorizer()
vectorizer.fit(df['text'])

x = vectorizer.transform(x)

In [ ]:
print(x)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 6904081 stored elements and shape (44898, 89868)>
  Coords	Values
  (0, 494)	0.023855463193607363
  (0, 641)	0.03041671753649333
  (0, 660)	0.026271653248021096
  (0, 1156)	0.1059863850562873
  (0, 1264)	0.03313796793674888
  (0, 2620)	0.060687808838532795
  (0, 4070)	0.03941996500744819
  (0, 4117)	0.040361900028891375
  (0, 8937)	0.0432820222696878
  (0, 9150)	0.04576946283492136
  (0, 9423)	0.13671725004918667
  (0, 11021)	0.019433127349709863
  (0, 11229)	0.09185383757561877
  (0, 12578)	0.1059863850562873
  (0, 13143)	0.06894872790604913
  (0, 13868)	0.027811593112511284
  (0, 14383)	0.03262091415845353
  (0, 14491)	0.06319836809964523
  (0, 14709)	0.09143507035246158
  (0, 15483)	0.0512374772591545
  (0, 15906)	0.10849391480762079
  (0, 16636)	0.043296219307691994
  (0, 17229)	0.036554887285286955
  (0, 17608)	0.022106355232944172
  (0, 17877)	0.033172620285351624
  :	:
  (44897, 5758)	0.10249762133423
  (44897, 14210)

In [ ]:
xtrain,xtest, ytrain,ytest = train_test_split(x,y, test_size = 0.2, stratify = y, random_state = 2)

In [ ]:
model = LogisticRegression()

In [ ]:
model.fit(xtrain, ytrain)

LogisticRegression()

In [ ]:
xtrain_prediction = model.predict(xtrain)
training_data_accuracy = accuracy_score(xtrain_prediction, ytrain)
print('Accuracy score of the training data : ', training_data_accuracy)

Accuracy score of the training data :  0.9910351355866139


In [ ]:
xtest_prediction = model.predict(xtest)
test_data_accuracy = accuracy_score(xtest_prediction, ytest)
print('Accuracy score of the test data : ', test_data_accuracy)

Accuracy score of the test data :  0.9879732739420936


In [ ]:
import joblib
from google.colab import files

joblib.dump(model, 'model.pkl')
joblib.dump(vectorizer, 'vectorizer.pkl')

['vectorizer.pkl']

In [ ]:
files.download('model.pkl')
files.download('vectorizer.pkl')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>